Note: 
May need to place the notebook into the idb-backend to run properly.

In [2]:
from __future__ import division, absolute_import, print_function

from pytz import timezone

import elasticsearch
import elasticsearch.helpers

from idb import config
from idb.helpers.logging import idblogger
from idb.helpers.conversions import fields, custom_mappings


# u is the uuid of the recordset that we wish to delete


In [3]:
u = "4dce41dc-2af6-448c-99e1-abfd3a9cc3e5"

In [4]:
serverlist = config.config["elasticsearch"]["servers"]



In [6]:
serverlist


[u'c18node2.acis.ufl.edu:9200',
 u'c18node6.acis.ufl.edu:9200',
 u'c18node10.acis.ufl.edu:9200',
 u'c18node12.acis.ufl.edu:9200',
 u'c18node14.acis.ufl.edu:9200']

In [7]:
def get_connection(**kwargs):
    kwargs.setdefault('hosts', config.config["elasticsearch"]["servers"])
    kwargs.setdefault('retry_on_timeout', True)  # this isn't valid until >=1.3                                                                                                                                                        
    kwargs.setdefault('sniff_on_start', False)
    kwargs.setdefault('sniff_on_connection_fail', False)
    kwargs.setdefault('max_retries', 10)
    kwargs.setdefault('timeout', 30)
    return elasticsearch.Elasticsearch(**kwargs)


In [8]:
conn = get_connection(hosts=serverlist)

In [9]:
r = conn.search(index="idigbio", doc_type="recordsets", _source=["etag"], body={
        "query": {
          "bool": {
            "must": [
              {
                "query_string": {
                  "default_field": "_id",
                  "query": u
                }
              }
            ],
            "must_not": [],
            "should": []
          }
        },
        "from": 0,
        "size": 10,
        "sort": [],
        "aggs": {}
      })

In [10]:
r['hits']

{u'hits': [], u'max_score': None, u'total': 0}

This from Nathan's example of deleting a mediarecord where we need the parent record:


In [33]:
#    parent = r['hits']['hits'][0]['_parent']
#    conn.delete(index='idigbio', id=u, parent=parent, doc_type='mediarecords')

In [34]:
conn.delete(index='idigbio', id=u, doc_type='recordsets')


NotFoundError: TransportError(404, u'{"found":false,"_index":"idigbio-2.9.3","_type":"recordsets","_id":"4dce41dc-2af6-448c-99e1-abfd3a9cc3e5","_version":1,"_shards":{"total":3,"successful":3,"failed":0}}')